# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 3993, done.
remote: Counting objects: 100% (1466/1466), done.
remote: Compressing objects: 100% (453/453), done.
remote: Total 3993 (delta 811), reused 1461 (delta 811), pack-reused 2527 (from 1)
Receiving objects: 100% (3993/3993), 167.98 MiB | 30.04 MiB/s, done.
Resolving deltas: 100% (2330/2330), done.
Updating files: 100% (374/374), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 30.1 MB/s eta 0:00:00


In [4]:
%cd /kaggle/working/RECsys_Challenge2024 
! python run_compile_all_cython.py

/kaggle/working/RECsys_Challenge2024
run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/11]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1252:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorizationI

In [5]:
from Utils.notebookFunctions import *
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil
import optuna
import json
import os
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit


K_PATH = '/kaggle/working/RECsys_Challenge2024'
GH_PATH = 'TrainedModels/WithoutKFCV/SLIM/OptimizingMAP'

np.random.seed(42)

## Import the repository

In [6]:
repo = get_repo_from_github(token)

Repository 'RECsys_Challenge2024' found.


In [7]:
config = {
    'model': 'SLIM_BPR',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_SLIM_BPR.db',
    'copy_prev_best_params': False,
    'tune_best_params': False,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [8]:
try:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}_Recommender/history_{config["model"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [9]:
URM_all_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1764607 stored elements and shape (35736, 38121)>

In [10]:
ICM_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2940040 stored elements and shape (38121, 94331)>

# Training

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [12]:
from Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python

def objective_function_slim_bpr(optuna_trial):

    recommender_instance = SLIM_BPR_Python(URM_train)

    full_hyperp = {
                    "topK": optuna_trial.suggest_int("topK", 0, 500),
                    "epochs": optuna_trial.suggest_int("epochs", 1, 50),
                    "lambda_i": optuna_trial.suggest_float("lambda_i", 1e-5, 1e-1, log=True),
                    "lambda_j": optuna_trial.suggest_float("lambda_j", 1e-5, 1e-1, log=True),
                    "learning_rate": optuna_trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
                }

    recommender_instance.fit(**full_hyperp)

    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)

    return result_df.loc[10]["MAP"]

In [13]:
if config['tune_parameters']:
    
    optuna_study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    optuna_study.optimize(objective_function_slim_bpr, n_trials=100)

[I 2024-12-15 23:38:38,251] A new study created in RDB with name: hyperparameters_tuning_SLIM_BPR


Epoch 1, Iteration 35736 in 10.44 seconds. Samples per second 3423.43
Epoch 2, Iteration 35736 in 7.65 seconds. Samples per second 4674.12
Epoch 3, Iteration 35736 in 6.20 seconds. Samples per second 5762.59
Epoch 4, Iteration 35736 in 5.36 seconds. Samples per second 6673.29
Epoch 5, Iteration 35736 in 4.55 seconds. Samples per second 7846.53
Epoch 6, Iteration 35736 in 4.18 seconds. Samples per second 8546.10
Epoch 7, Iteration 35736 in 3.82 seconds. Samples per second 9353.59
Epoch 8, Iteration 35736 in 3.61 seconds. Samples per second 9887.14
Epoch 9, Iteration 35736 in 3.52 seconds. Samples per second 10138.65
Epoch 10, Iteration 35736 in 3.45 seconds. Samples per second 10363.84
Epoch 11, Iteration 35736 in 3.32 seconds. Samples per second 10760.85
Epoch 12, Iteration 35736 in 3.38 seconds. Samples per second 10569.74
Epoch 13, Iteration 35736 in 3.26 seconds. Samples per second 10960.49
Epoch 14, Iteration 35736 in 3.14 seconds. Samples per second 11397.73
Epoch 15, Iteration 35

[I 2024-12-15 23:42:09,441] Trial 0 finished with value: 0.030346353487313602 and parameters: {'topK': 80, 'epochs': 21, 'lambda_i': 0.002597835793983858, 'lambda_j': 0.0001362653966245703, 'learning_rate': 0.00029026429634871503}. Best is trial 0 with value: 0.030346353487313602.


Epoch 1, Iteration 35736 in 13.07 seconds. Samples per second 2735.10
Epoch 2, Iteration 35736 in 13.50 seconds. Samples per second 2648.08
Epoch 3, Iteration 35736 in 8.08 seconds. Samples per second 4420.53
Epoch 4, Iteration 35736 in 6.51 seconds. Samples per second 5489.07
Epoch 5, Iteration 35736 in 5.46 seconds. Samples per second 6543.34
Epoch 6, Iteration 35736 in 4.95 seconds. Samples per second 7214.57
Epoch 7, Iteration 35736 in 4.49 seconds. Samples per second 7958.29
Epoch 8, Iteration 35736 in 4.22 seconds. Samples per second 8461.89
Epoch 9, Iteration 35736 in 3.94 seconds. Samples per second 9080.22
Epoch 10, Iteration 35736 in 3.69 seconds. Samples per second 9679.96
Epoch 11, Iteration 35736 in 3.64 seconds. Samples per second 9820.20
Epoch 12, Iteration 35736 in 3.48 seconds. Samples per second 10276.26
Epoch 13, Iteration 35736 in 3.35 seconds. Samples per second 10679.07
Epoch 14, Iteration 35736 in 3.31 seconds. Samples per second 10789.92
Epoch 15, Iteration 3573

[I 2024-12-15 23:46:33,645] Trial 1 finished with value: 0.030389690455007883 and parameters: {'topK': 210, 'epochs': 17, 'lambda_i': 0.0017687051598126319, 'lambda_j': 0.008016444952382434, 'learning_rate': 0.03126559091252736}. Best is trial 1 with value: 0.030389690455007883.


...

[I 2024-12-16 06:49:48,943] Trial 97 finished with value: 0.035152999908582276 and parameters: {'topK': 69, 'epochs': 24, 'lambda_i': 0.07720629734877726, 'lambda_j': 0.00012334144974821288, 'learning_rate': 0.056656118490598845}. Best is trial 94 with value: 0.03867352958882535.


Epoch 1, Iteration 35736 in 9.98 seconds. Samples per second 3581.93
Epoch 2, Iteration 35736 in 7.26 seconds. Samples per second 4921.86
Epoch 3, Iteration 35736 in 5.93 seconds. Samples per second 6026.26
Epoch 4, Iteration 35736 in 5.05 seconds. Samples per second 7074.49
Epoch 5, Iteration 35736 in 4.47 seconds. Samples per second 7989.80
Epoch 6, Iteration 35736 in 4.05 seconds. Samples per second 8834.31
Epoch 7, Iteration 35736 in 3.77 seconds. Samples per second 9486.20
Epoch 8, Iteration 35736 in 3.70 seconds. Samples per second 9655.56
Epoch 9, Iteration 35736 in 3.45 seconds. Samples per second 10362.46
Epoch 10, Iteration 35736 in 3.37 seconds. Samples per second 10610.80
Epoch 11, Iteration 35736 in 3.31 seconds. Samples per second 10796.80
Epoch 12, Iteration 35736 in 3.24 seconds. Samples per second 11033.45
Epoch 13, Iteration 35736 in 3.25 seconds. Samples per second 10988.58
Epoch 14, Iteration 35736 in 3.21 seconds. Samples per second 11133.70
Epoch 15, Iteration 357

[I 2024-12-16 06:54:13,183] Trial 98 finished with value: 0.040251465450157724 and parameters: {'topK': 10, 'epochs': 29, 'lambda_i': 0.09900111245281501, 'lambda_j': 1.234691796411495e-05, 'learning_rate': 0.08668948504743702}. Best is trial 98 with value: 0.040251465450157724.


Epoch 1, Iteration 35736 in 10.17 seconds. Samples per second 3512.33
Epoch 2, Iteration 35736 in 7.39 seconds. Samples per second 4837.65
Epoch 3, Iteration 35736 in 5.80 seconds. Samples per second 6163.30
Epoch 4, Iteration 35736 in 4.98 seconds. Samples per second 7170.69
Epoch 5, Iteration 35736 in 4.43 seconds. Samples per second 8071.62
Epoch 6, Iteration 35736 in 4.09 seconds. Samples per second 8744.92
Epoch 7, Iteration 35736 in 3.88 seconds. Samples per second 9203.78
Epoch 8, Iteration 35736 in 3.73 seconds. Samples per second 9571.67
Epoch 9, Iteration 35736 in 3.57 seconds. Samples per second 10022.93
Epoch 10, Iteration 35736 in 3.44 seconds. Samples per second 10401.81
Epoch 11, Iteration 35736 in 3.31 seconds. Samples per second 10794.17
Epoch 12, Iteration 35736 in 3.23 seconds. Samples per second 11061.15
Epoch 13, Iteration 35736 in 3.19 seconds. Samples per second 11199.87
Epoch 14, Iteration 35736 in 3.09 seconds. Samples per second 11579.15
Epoch 15, Iteration 35

[I 2024-12-16 06:58:34,816] Trial 99 finished with value: 0.038075866728204295 and parameters: {'topK': 13, 'epochs': 29, 'lambda_i': 0.09977274182397447, 'lambda_j': 3.4280869624778604e-05, 'learning_rate': 0.048302392186519064}. Best is trial 98 with value: 0.040251465450157724.


## Some optuna visualizations on recommender parameters

In [14]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_slice(optuna_study)
fig.show()

In [15]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_param_importances(optuna_study)
fig.show()

## Let's train the recommender with best parameter values

In [16]:
if config['tune_best_params']:

    if config['tune_parameters']:
        best_params = optuna_study.best_trial.params
    else: 
        with open(f'{K_PATH}/{GH_PATH}/{config["model"]}_Recommender/best_params_{config["model"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

    best_params = study.best_trial.params
    
    recommender_instance = SLIM_BPR_Python(URM_train + URM_validation)
    recommender_instance.fit(**best_params)

# Testing

Create the recommendations for the submission. 

In [17]:
if config['tune_best_params']:

    data_target_users_test = pd.read_csv('/kaggle/input/recommender-system-2024-challenge-polimi/data_target_users_test.csv')
    create_submission(data_target_users_test, recommender_instance, f'/kaggle/working/submission_{config["model"]}.csv')

# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [18]:
if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}.json', 'w') as params_file:
        json.dump(optuna_study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}.json', 
            f'{GH_PATH}/{config["model"]}_Recommender/best_params_{config["model"]}.json', 
            f'{config["model"]} tuning results (from kaggle notebook)',
            repo
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}_Recommender/'\
        f'best_params_{config["model"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}.json'
    )

File 'TrainedModels/WithoutKFCV/SLIM/OptimizingMAP/SLIM_BPR_Recommender/best_params_SLIM_BPR.json' created successfully.


Save the history of the tuned model.

In [19]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'{GH_PATH}/{config["model"]}_Recommender/history_{config["model"]}.db',
        f'Tuning {config["model"]} db updated results (from kaggle notebook)',
        repo
    )

File 'TrainedModels/WithoutKFCV/SLIM/OptimizingMAP/SLIM_BPR_Recommender/history_SLIM_BPR.db' created successfully.


Save the best trained model and its submission.

In [20]:
if config['save_github'] and config['tune_best_params']: 
    upload_file(
                f'/kaggle/working/submission_{config["model"]}.csv', 
                f'{GH_PATH}/{config["model"]}_Recommender/Submission/submission_{config["model"]}.csv', 
                f'New {config["model"]} submission (from kaggle notebook)',
                repo
            )